# 7.4 PDE-Constrained Shape Optimization in NGSolve

We want to solve the PDE-constrained shape optimization problem
\begin{equation}
            \underset{\Omega\subset \mathsf{D}}{\mbox{min}} \; J(u) := \int_\Omega |u-u_d|^q \; dx, \quad q\ge 2
\end{equation}
 subject to that $(\Omega,u)$ satisfy
 \begin{equation}
           \int_\Omega \nabla u \cdot \nabla v \; dx = \int_\Omega f v \; dx \; \quad \text{ for all } v \in H_0^1(\Omega),
\end{equation}
where $\Omega \subset \mathbb R^2$ for given $u_d, f \in C^1(\mathbb R^2)$.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
from ngsolve.solvers import *

In [ ]:
geo = SplineGeometry()
geo.AddCircle(c=(0.5,0.5), r=0.5, bc = 'circle')
#geo.AddRectangle( (0.2, 0.2), (0.8, 0.8), bcs = ('b','r','t','l'))
mesh = Mesh(geo.GenerateMesh(maxh = 0.1))

In [ ]:
#given data of our problem (chosen such that \Omega^* = [0,1]^2 is the optimal shape)
f = CoefficientFunction(2*y*(1-y)+2*x*(1-x))
ud = x*(1-x)*y*(1-y)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

In [ ]:
fes = H1(mesh, order=2, dirichlet=".*")
gfu = GridFunction(fes)
scene_u = Draw (gfu, mesh, "state")

gfp = GridFunction(fes)
scene_p = Draw (gfp, mesh, "adjoint")

Note that (for linear problems) the operator on the left hand side of the adjoint equation is just the transpose of the state operator.

### Automatic Shape Differentiation

The formula for the shape derivative was derived as the partial derivative of the perturbed Lagrangian (brought back to the original domain):
\begin{align*}
    d\mathcal J(\Omega; X) = \frac{\partial}{\partial t} \left( \int_\Omega |u - u_d^t|^q  \,\mbox{det}(F_t) \mbox{d} x 
         +  \int_{\Omega} (F_t^{-\top}\nabla u) \cdot (F_t^{-\top} \nabla p)\,  \mbox{det}(F_t) \, dx - \int_{\Omega}f^t p  \,\mbox{det}(F_t) \,dx \right)|_{t=0} 
\end{align*}
where 
<ul>
    <li>   $T_t(x)=x+tX(x)=y$ 
    <li> $F_t = DT_t = I+t DX$
    <li>  $u_d^t = u_d \circ T_t$
    <li> $f^t = f \circ T_t$
</ul>

The integrand depends on the parameter $t$ only via $F_t$ and $T_t$. We define the Lagrangian in this form, involving a parameter t that has the value 0.

In [ ]:
VEC = H1(mesh, order=2, dim=2)
PHI, X = VEC.TnT()

#define symbolic parameter t=0 in order to allow differentiation w.r.t. t
t = Parameter(0)
F = Id(2) + t * grad(X).trans

def Equation(u,v):
    return ( (Inv(F.trans)*grad(u))*(Inv(F.trans)*grad(v))-f*v)*Det(F)

q=6
def CostAuto(u): 
    return (u-ud)**q*Det(F)

def CostAuto2(u): 
    return (u-ud)**q

Lagrangian = CostAuto(gfu) + Equation(gfu,gfp)

### State equation

Equation can also be used to define the bilinear form. The following defines left and right hand side of the PDE in a "BilinearForm":

In [ ]:
u, v = fes.TnT()

aAuto = BilinearForm(fes, symmetric=True)
aAuto += Equation(u,v)*dx

Now the PDE can be conveniently solved by calling Newton's method (which terminates after one iteration since the PDE is linear)

In [ ]:
gfu.vec[:]=0
Newton(aAuto,gfu,freedofs=fes.FreeDofs())
scene_u.Redraw()

### Adjoint equation

We set up the adjoint equation
\begin{align*}
    \mbox{Find } p \in H_0^1(\Omega): \int_\Omega \nabla w \cdot \nabla p \, \mbox dx = - \partial_u J(u)(w) \quad \text{ for all } w \in H_0^1(\Omega)
\end{align*}
where $u$ is the solution to the state equation. For $J(u) = \int_\Omega |u-u_d|^2 \mbox dx$, we get
\begin{align*}
    \partial_u J(u)(w) = 2 \int_\Omega (u-u_d)w \,\mbox dx.
\end{align*}
However, we can also use the Diff(...) command:

In [ ]:
p, w = fes.TnT()

fadjoint = LinearForm(fes)
fadjoint += -1*(CostAuto(gfu)*dx).Diff(gfu,w)

In [ ]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - aAuto.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = aAuto.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [ ]:
fadjoint.Assemble()
SolveAdjointEquation()
scene_p.Redraw()

### Automatic Shape Differentiation

Denoting the integrand by $G^{u,p}$, the shape derivative is given by
\begin{align*}
     d\mathcal J(\Omega; X) =& \left( \left. \frac{\partial G^{u,p}}{\partial t} + \frac{d  G^{u,p}}{dy} \cdot \frac{d T_t}{dt}\right)\right\rvert_{t=0} \\
     =& \left.  \frac{\partial G^{u,p}}{\partial t}\right\rvert_{t=0} + \frac{d  G^{u,p}}{dy} \cdot X
\end{align*}

In [ ]:
dJOmegaAuto = LinearForm(VEC)
dJOmegaAuto += Lagrangian.Diff(t) * dx
dJOmegaAuto += (Lagrangian.Diff(x, X[0]) + Lagrangian.Diff(y, X[1])) * dx

In [ ]:
b = BilinearForm(VEC)
b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

gfX = GridFunction(VEC)

# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
scene=Draw(gfset,mesh,"gfset")

In [ ]:
def SolveDeformationEquationAuto():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmegaAuto.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [ ]:
b.Assemble()
dJOmegaAuto.Assemble()
SolveDeformationEquationAuto()
Draw(-gfX, mesh, "-gfX")

In [ ]:
print('Cost at initial design', Integrate (CostAuto2(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

In [ ]:
mesh.SetDeformation(gfset)
gfu.vec[:]=0
Newton(aAuto, gfu, fes.FreeDofs())
print('Cost at new design', Integrate (CostAuto2(gfu), mesh))
scene.Redraw()

Thus, the user has to enter the PDE (in its transformed form) only once.

Finally, let us again run the full algorithm:

In [ ]:
#reset to and solve for initial configuration
scene_u = Draw(gfu)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
gfu.vec[:]=0
Newton(aAuto, gfu, fes.FreeDofs())

LineSearch = True

iter_max = 600
Jold = Integrate(CostAuto2(gfu), mesh)
converged = False
for k in range(iter_max):
    print('cost at iteration', k, ': ', Jold)
    mesh.SetDeformation(gfset)
    scene_u.Redraw()
    
    gfu.vec[:]=0
    Newton(aAuto, gfu, fes.FreeDofs(), printing = False)
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmegaAuto.Assemble()
    SolveDeformationEquationAuto()

    scale = 0.01 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    
    Jnew = Integrate(CostAuto2(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            gfu.vec[:]=0
            Newton(aAuto, gfu, fes.FreeDofs(), printing = False)
            Jnew = Integrate(CostAuto2(gfu), mesh)
    
    if converged==True:
        break
    Jold = Jnew

    Redraw(blocking=True)